<a href="https://colab.research.google.com/github/ZHANJINGCHUN/metrics_compute/blob/main/NIQE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-image

In [ ]:
import os
import numpy as np
from skimage import io, color, img_as_float
from skimage.filters import sobel
from scipy.ndimage import convolve
from scipy.stats import norm

In [ ]:
def compute_niqe(image):
    # Convert the image to grayscale
    image_gray = color.rgb2gray(image)

    # Convert the image to float
    image_float = img_as_float(image_gray)

    # Compute gradient magnitude
    grad_x = sobel(image_float, axis=0)
    grad_y = sobel(image_float, axis=1)
    gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)

    # Compute local statistics
    window_size = 32
    local_means = convolve(image_float, np.ones((window_size, window_size)) / window_size**2)
    local_vars = convolve(image_float**2, np.ones((window_size, window_size)) / window_size**2) - local_means**2

    # Compute NIQE features
    niqe_features = [
        np.mean(gradient_magnitude),
        np.mean(local_means),
        np.mean(local_vars),
        np.std(gradient_magnitude),
        np.std(local_means),
        np.std(local_vars),
        norm.ppf(np.mean(gradient_magnitude)),
        norm.ppf(np.mean(local_means)),
        norm.ppf(np.mean(local_vars)),
        norm.ppf(np.std(gradient_magnitude)),
        norm.ppf(np.std(local_means)),
        norm.ppf(np.std(local_vars))
    ]

    # Fit a Gaussian mixture model
    niqe_mean = np.mean(niqe_features)
    niqe_std = np.std(niqe_features)
    niqe_normalized = (niqe_features - niqe_mean) / niqe_std
    niqe_score = np.mean(niqe_normalized)

    return niqe_score

In [ ]:
def main(folder_path):
    total_niqe = 0
    num_images = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            image_path = os.path.join(folder_path, filename)
            # Load an image
            image = io.imread(image_path)
            # Compute NIQE score
            niqe_score = compute_niqe(image)
            print(f"{filename}: NIQE Score = {niqe_score}")
            total_niqe += niqe_score
            num_images += 1

    if num_images > 0:
        average_niqe = total_niqe / num_images
        print(f"Average NIQE Score: {average_niqe}")
    else:
        print("No images found in the folder.")

In [ ]:
if __name__ == "__main__":
    folder_path = '/content/drive/MyDrive/DATASETS/LOLv1/eval15/high/'
    main(folder_path)

146.png: NIQE Score = -5.551115123125783e-17
669.png: NIQE Score = 0.0
79.png: NIQE Score = -5.551115123125783e-17
22.png: NIQE Score = 1.1102230246251565e-16
778.png: NIQE Score = -1.850371707708594e-17
179.png: NIQE Score = 5.551115123125783e-17
55.png: NIQE Score = -7.401486830834377e-17
1.png: NIQE Score = -7.401486830834377e-17
547.png: NIQE Score = 7.401486830834377e-17
493.png: NIQE Score = -1.4802973661668753e-16
748.png: NIQE Score = 1.850371707708594e-17
780.png: NIQE Score = 1.850371707708594e-17
665.png: NIQE Score = -9.251858538542972e-17
23.png: NIQE Score = -5.551115123125783e-17
111.png: NIQE Score = 1.6653345369377348e-16
Average NIQE Score: -8.635067969306772e-18
